# JLCPCB Local Database Search

Requires parts database to already be downloaded and unzipped ```jlcpcb-components.sqlite3```

[Setup Python Virtual Environment in VSCode](/venv_setup.md)

In [16]:
import sqlite3
import json

conn = sqlite3.connect(r"jlcpcb-components.sqlite3")
cur = conn.cursor()

cur.execute("PRAGMA temp_store = MEMORY")  # Store temporary tables and indices in memory for faster access
cur.execute("PRAGMA mmap_size = 2147483648")  # Set the maximum memory map size to 2 GiB

In [18]:
# Search terms
search_terms = [
    "SARA-R520M10",
    "SARA-R520",
    "BG950A-GL",
    "GM02SP",
    "L710",
    "HL7810",
    "HL7812",
    "L710HG",
    "SKY66431",
    "SARA-R510",
    "SARA-R422",
    "SIM7070G",
    "BG770A-GL",
    "SIM7080G",
    "RYZ024A",
    "ME310G1-W3",
    "ME910G1-W3",
    "TX62-W",
    "TPM540",
    "MA510-GL",
    "TX82-W",
    "EXS62-W",
    "RYZ014A",
    "EXS82-W",
    "BG95",
    "BG77",
    "ME910C1-P1",
    "GM01Q",
    "HL7800-M",
    "WP7702",
    "SARA-R410M",
    "ME910C1-WW",
    "ML865C1-NA",
    "ME910C1-NA",
    "nRF9160",
]

query = """
    SELECT c.price, c.stock, f.*
    FROM components c
    JOIN components_fts f ON c.lcsc = f.lcsc
    WHERE f.mfr LIKE ? OR f.description LIKE ? OR f.datasheet LIKE ? OR f.lcsc LIKE ?
"""

rows = []
for term in search_terms:
    # Execute the query
    cur.execute(
        query,
        (f"%{term}%",) * 4,
    )

    # Fetch the results and append to rows
    rows.extend(cur.fetchall())

# Sort the rows based on the price
rows.sort(
    key=lambda row: json.loads(row[0])[0]["price"],
    reverse=True,
)

# Print the results
for i, row in enumerate(rows):
    price_data = json.loads(row[0])
    print(f"{row[3]}")
    print(f"Price: {round(price_data[0]['price'], 2)}USD")
    print(f"Stock: {row[1]}")
    print(f"Package: {row[4]}")
    print(f"Description: {row[5]}")
    print(f"https://jlcpcb.com/partdetail/C{row[2]}")
    print()

SIM7070G
Price: 13.32USD
Stock: 50
Package: LCC-68(24x24)
Description: LCC-68(24x24)  RF Modules ROHS
https://jlcpcb.com/partdetail/C2892847

SIM7080G
Price: 11.55USD
Stock: 352
Package: SMD,15.7x17.6mm
Description: SMD,15.7x17.6mm  2G/3G/4G/5G Modules ROHS
https://jlcpcb.com/partdetail/C2943992

BG95M3LA-64-SGNS
Price: 11.03USD
Stock: 298
Package: SMD,19.9x23.6mm
Description: 射频模块, USB 2.0接口
https://jlcpcb.com/partdetail/C2960993

SIM7080G-M
Price: 9.27USD
Stock: 154
Package: SMD,15.7x17.6mm
Description: SMD,15.7x17.6mm  2G/3G/4G/5G Modules ROHS
https://jlcpcb.com/partdetail/C18548266

BG95M1LA-64-SGNS
Price: 8.65USD
Stock: 13
Package: -
Description: Transceiver Module Surface Mount
https://jlcpcb.com/partdetail/C5366871

EL7104CSZ-T7
Price: 6.39USD
Stock: 37
Package: SOIC-8
Description: Low Edge 1 MOSFET 4A 7.5ns 10ns 4.5V~16V 4A SOIC-8  Gate Drivers ROHS
https://jlcpcb.com/partdetail/C2676930

ICL7107CM44Z
Price: 5.32USD
Stock: 5
Package: MQFP-44(10x10)
Description: MQFP-44(10x10)  

In [19]:
# Close the database connection
conn.close()